In [ ]:
import gurobipy as gp
import numpy as np
import math
import copy

In [ ]:
def previous(i, j, J):
    before = copy.deepcopy(J)
    before = [i] + before
    if j != 0:
        before.remove(j)
    return before


def later(i, j, bar_J):
    after = copy.deepcopy(bar_J)
    if i != j:
        after.remove(j)
    return after

# 导入数据
np.set_printoptions(suppress=True)    # 取消numpy打印的科学计数法


# cost 矩阵的第一索引位置是0 默认为虚拟设施
root = '../data/SnyderData/15nodes/'
cost = np.loadtxt(root+'cost.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

dmd = np.loadtxt(root+'dmd.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

fc = np.loadtxt(root+'fc.csv',  # 相对路径下的csv文件
                  dtype=None,         # 数据类型默认
                  encoding='UTF-8',   # 注意此文件为UTF-8格式且取消BOM
                  delimiter=',')      # 分隔符

In [10]:
rou = 0.05 # 损坏概率参数
max_visit_num = 3 # 客户的最大尝试次数

for rou in [0.05, 0.1, 0.2, 0,3]:
    for max_visit_num in [2,3,4,5,6,7,9,10]:
        print('rho = %.2f, R = %d\t' %(rou, max_visit_num))
        q = rou * np.exp(-fc/200000) # 损坏概率
        q[0] = 1

        cus_num = len(dmd) # 客户数
        node_num = cost.shape[0] # 节点数（虚拟 设施 客户）
        fac_num = node_num - cus_num - 1 # 设施数

        # 集合设置
        J = [j for j in range(1, fac_num+1)] # 设施集合
        I = [i for i in range(fac_num+1, node_num)] # 客户集合
        bar_J = [j for j in range(0, fac_num+1)] # 设施拓展集合
        R = [r for r in range(1,max_visit_num)] # 等级

        # 常数集合
        lmd = {i : dmd[i-fac_num-1] for i in I} # lambda需求
        c = {(i,j) : cost[i,j] for i in bar_J+I for j in bar_J} # 价格
        f = {j : fc[j] for j in J} # 建设成本

        m = gp.Model()
        y = m.addVars((j for j in J), vtype = gp.GRB.BINARY, name = 'y')
        x = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), vtype = gp.GRB.BINARY,name = 'x') # 弧(j,j_p)属于客户i
        p = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS,name = 'p') # 属于客户i的弧(j,j_p)的概率

        item_1 = gp.quicksum(f[j] * y[j] for j in J)
        item_2 = gp.quicksum((lmd[i] * c[k,j] * p[i,k,j] * x[i,k,j]) for i in I for j in bar_J for k in previous(i,j,J))
        m.setObjective(item_1 + item_2)

        m.addConstrs((gp.quicksum(x[i,k,j] for k in previous(i,j,J)) <= y[j] for i in I for j in J), name='assign2open')
        m.addConstrs((gp.quicksum(x[i,i,j] for j in later(i,i,bar_J)) == 1 for i in I), name = 'flowin')
        m.addConstrs((gp.quicksum(x[i,j,0] for j in previous(i,0,J)) == 1 for i in I), name = 'flowout')
        m.addConstrs((gp.quicksum(x[i,j,k] for k in later(i,j,bar_J)) == gp.quicksum(x[i,k,j] for k in previous(i,j,J)) for i in I for j in J), name = 'balance')
        m.addConstrs((p[i,i,j] == x[i,i,j] for i in I for j in later(i,i,bar_J)), name = 'probinit')
        m.addConstrs(((q[j] * gp.quicksum(x[i,k,j] * p[i,k,j] for k in previous(i,j,J)) == p[i,j,j_p]) for i in I for j in J for j_p in later(i,j,bar_J)), name = 'probbalance')
        m.addConstrs(((gp.quicksum(x[i,j,k] for j in J+[i] for k in later(i,j,bar_J))) <= max_visit_num for i in I), name = 'maxtry')
        m.Params.OutputFlag = 0
        m.Params.timeLimit = 1000
        m.optimize()

        print('%.2f\t%.2f\t%.2f\t' %(m.objVal, m.ObjBound, m.RunTime), end='')


        m = gp.Model()
        y = m.addVars((j for j in J), vtype = gp.GRB.BINARY, name = 'y')
        x = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), vtype = gp.GRB.BINARY,name = 'x') # 弧(j,j_p)属于客户i
        p = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS,name = 'p') # 属于客户i的弧(j,j_p)的概率
        w = m.addVars(((i, j, j_p) for i in I for j in J+[i] for j_p in later(i,j,bar_J)), lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS,name = 'w') # 期望价格
        item_1 = gp.quicksum(f[j] * y[j] for j in J)
        item_2 = gp.quicksum((lmd[i] * c[k,j] * w[i,k,j]) for i in I for j in bar_J for k in previous(i,j,J))
        m.setObjective(item_1 + item_2)
        m.addConstrs((gp.quicksum(x[i,k,j] for k in previous(i,j,J)) <= y[j] for i in I for j in J), name='assign2open')
        m.addConstrs((gp.quicksum(x[i,i,j] for j in later(i,i,bar_J)) == 1 for i in I), name = 'flowin')
        m.addConstrs((gp.quicksum(x[i,j,0] for j in previous(i,0,J)) == 1 for i in I), name = 'flowout')
        m.addConstrs((gp.quicksum(x[i,j,k] for k in later(i,j,bar_J)) == gp.quicksum(x[i,k,j] for k in previous(i,j,J)) for i in I for j in J), name = 'balance')
        m.addConstrs((p[i,i,j] == x[i,i,j] for i in I for j in later(i,i,bar_J)), name = 'probinit')
        m.addConstrs(((q[j] * gp.quicksum(w[i,k,j] for k in previous(i,j,J)) == p[i,j,j_p]) for i in I for j in J for j_p in later(i,j,bar_J)), name = 'probbalance')
        m.addConstrs(((gp.quicksum(x[i,j,k] for j in J+[i] for k in later(i,j,bar_J))) <= max_visit_num for i in I), name = 'maxtry')
        m.addConstrs((w[i,j,k] <= p[i,j,k] for i in I for j in J+[i] for k in later(i,j,bar_J)), name='u1b')
        m.addConstrs((w[i,j,k] <= x[i,j,k] for i in I for j in J+[i] for k in later(i,j,bar_J)), name='u2b')
        m.addConstrs((w[i,j,k] >= p[i,j,k] + x[i,j,k] -1 for i in I for j in J+[i] for k in later(i,j,bar_J)), name='lb')
        m.Params.OutputFlag = 0
        m.Params.timeLimit = 1000
        m.optimize()

        print('%.2f\t%.2f\t%.2f' %(m.objVal, m.ObjBound, m.RunTime))





rho = 0.05, R = 2	
1184928.86	1184928.86	0.03	1184928.86	1184928.86	0.04
rho = 0.05, R = 3	
664900.02	664900.02	1.97	664900.02	664900.02	1.44
rho = 0.05, R = 4	
644206.22	644206.22	3.51	644206.22	644206.22	2.13
rho = 0.05, R = 5	
643430.51	643430.51	3.03	643430.36	643428.97	2.59
rho = 0.05, R = 6	
643404.86	643402.27	1.94	643400.97	643396.37	2.42
rho = 0.05, R = 7	
643401.31	643399.27	2.42	643402.62	643399.56	2.14
rho = 0.05, R = 9	
643402.20	643401.83	2.77	643402.13	643402.13	2.45
rho = 0.05, R = 10	
643401.54	643397.45	3.27	643401.47	643399.54	1.74
rho = 0.10, R = 2	
1736825.35	1736825.35	0.02	1736825.35	1736825.35	0.03
rho = 0.10, R = 3	
777800.34	777800.34	5.99	777800.34	777800.34	6.83
rho = 0.10, R = 4	
698526.23	698526.22	221.78	
Interrupt request received
698526.23	694732.82	40.01
rho = 0.10, R = 5	
